<a href="https://colab.research.google.com/github/BrianNjau/AI-GROUPWORK/blob/master/GroupProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine Learning Group Work

Our project is a Spam Detection and Deep NLP Model. 

This model aims to curb spam by text message classification.  

Group Members


---



*   Brian Njau **110372** 
*   Michael Mukiri **110628**


In [ ]:
import numpy as np
import pandas as pd

import os 
print(os.listdir("../"));


['root', 'home', 'etc', 'proc', 'media', 'usr', 'lib', 'sys', 'lib64', 'bin', 'srv', 'run', 'boot', 'tmp', 'sbin', 'dev', 'mnt', 'var', 'opt', '.dockerenv', 'tools', 'datalab', 'tensorflow-1.15.2', 'content', 'lib32']


In [ ]:
data = pd.read_csv("./spamdata.csv")

In [ ]:
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
data["Category"] = [1 if each == "spam" else 0 for each in data["Category"]] 

In [ ]:
data.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:

#Adding punctuation in row 1
import re 
nlp_data = str(data.iloc[2,:])
nlp_data = re.sub("[^a-zA-Z]"," ", nlp_data)


In [ ]:
nlp_data = nlp_data.lower()

In [ ]:
import nltk as nlp
nlp_data =  nlp.word_tokenize(nlp_data)


In [ ]:
lemma = nlp.WordNetLemmatizer()
nlp_data = [lemma.lemmatize(word) for word in nlp_data]


In [ ]:
nlp_data = " ".join(nlp_data)

**NLP**


In [ ]:
import nltk as nlp
import re
description_list = []
for description in data["Message"]:
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower()
    description = nlp.word_tokenize(description)
    #description = [ word for word in description if not word in set(stopwords.words("english"))]
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
max_features = 3000 #We use the most common word
count_vectorizer = CountVectorizer(max_features = max_features, stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()
print("the most using {} words: {}".format(max_features,count_vectorizer.get_feature_names()))

the most using 3000 words: ['aah', 'aathi', 'abi', 'ability', 'abiola', 'abj', 'able', 'absolutly', 'abt', 'abta', 'aburo', 'ac', 'academic', 'acc', 'accept', 'access', 'accident', 'accidentally', 'accordingly', 'account', 'ache', 'acl', 'aco', 'acted', 'acting', 'action', 'activate', 'active', 'activity', 'actor', 'actual', 'actually', 'ad', 'adam', 'add', 'addamsfa', 'added', 'addicted', 'addie', 'address', 'admin', 'administrator', 'admirer', 'admit', 'adore', 'adoring', 'adult', 'advance', 'adventure', 'advice', 'advise', 'ae', 'aeronautics', 'aeroplane', 'affair', 'affection', 'afraid', 'aft', 'afternoon', 'aftr', 'ag', 'agalla', 'age', 'agent', 'ago', 'agree', 'ah', 'aha', 'ahead', 'ahmad', 'aid', 'aight', 'ain', 'aint', 'air', 'airport', 'airtel', 'aiya', 'aiyah', 'aiyar', 'aiyo', 'aj', 'aka', 'al', 'alaipayuthe', 'album', 'alcohol', 'alert', 'alex', 'alfie', 'algarve', 'ali', 'alive', 'allah', 'allow', 'allowed', 'alright', 'alrite', 'alwys', 'amazing', 'american', 'amp', 'amt'

In [ ]:
#We separate our data is train and test
y = data.iloc[:,0].values   # male or female classes
x = sparce_matrix
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)


In [ ]:
#We make model for predict
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train,y_train)
print("the accuracy of our model: {}".format(nb.score(x_test,y_test)))

the accuracy of our model: 0.8763440860215054


In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter = 200)
lr.fit(x_train,y_train)
print("our accuracy is: {}".format(lr.score(x_test,y_test)))

our accuracy is: 0.974910394265233


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)
#print('Prediction: {}'.format(prediction))
print('With KNN (K=3) accuracy is: ',knn.score(x_test,y_test))

With KNN (K=3) accuracy is:  0.942652329749104


In [ ]:
x_test = x_test.reshape(558,3000,1)

In [ ]:
x_train = x_train.reshape(5014,3000,1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics=["accuracy"])

regressor.fit(x_test, y_test, epochs = 3, batch_size = 32)


Epoch 1/3
18/18 [==============================] - 120s 6s/step - loss: 0.1160 - accuracy: 0.8692
Epoch 2/3
18/18 [==============================] - 113s 6s/step - loss: 0.1143 - accuracy: 0.8692
Epoch 3/3
18/18 [==============================] - 113s 6s/step - loss: 0.1141 - accuracy: 0.8692
